In [1]:
!git clone https://github.com/MockaWolke/featout_hackathon
%cd featout_hackathon
!bash load_data.sh
!pip install -e .
!pip install captum

Cloning into 'featout_hackathon'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 145 (delta 20), reused 44 (delta 14), pack-reused 91
Receiving objects: 100% (145/145), 32.43 MiB | 11.88 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/featout_hackathon
--2023-09-14 17:09:51--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.36.48.252, 2600:1413:b000:793::317f, 2600:1413:b000:78a::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.36.48.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   102MB/s    in 6.5s    

2023-09-14 17:09:57 (121 MB/s) - ‘kagglecatsanddogs_534

In [2]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import csv
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from featout_exp.utils import load_data, load_model, load_data_raw_dataset
from featout_exp.old_featout import *

In [3]:
BATCHSIZE = 32
NUM_EPOCHS = 10
EXP_NAME = "Standard_Featout"
TESTING = False
BLUR_METHOD = zero_out
# algorithm to derive the model's attention
ATTENTION_ALGORITHM = simple_gradient_saliency

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train_dataset = load_data_raw_dataset("train")
test_dataset = load_data_raw_dataset("test")

train_featout = Featout(train_dataset, "test_images", do_plotting= False, device= device)

train_loader = torch.utils.data.DataLoader(
    train_featout, batch_size= 32, shuffle=True, num_workers=0
)

test_loader = load_data("test", 32, False)


EXP_PATH = os.path.join("logs",EXP_NAME)

os.makedirs(EXP_PATH, exist_ok=True)
# Initialize TensorBoard writer
writer = SummaryWriter(EXP_PATH)

# Prepare a CSV file for logging
csv_file = open(os.path.join(EXP_PATH, "logs.csv"), "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Test Loss", "Test Accuracy"])

57

In [5]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    if epoch == 1:
      train_loader.dataset.start_featout(
          model,
          blur_method=BLUR_METHOD,
          algorithm=ATTENTION_ALGORITHM,
      )
    for step, (inputs, labels) in enumerate(bar):


        if TESTING == True and step > 2:
            break

        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = corrects.double() / total

    # Evaluate on test data
    model.eval()
    test_running_loss = 0.0
    test_corrects = 0
    test_total = 0

    with torch.no_grad():
        for step,  (inputs, labels) in enumerate(tqdm( test_loader)):
            if TESTING == True and step > 2:
                 break

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)
            test_total += labels.size(0)

    test_loss = test_running_loss / test_total
    test_acc = test_corrects.double() / test_total

    # Log metrics to TensorBoard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Loss/test", test_loss, epoch)
    writer.add_scalar("Accuracy/test", test_acc, epoch)

    # Log metrics to CSV
    csv_writer.writerow([epoch + 1, train_loss, train_acc.item(), test_loss, test_acc.item()])
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")

csv_file.close()
writer.close()

torch.save(model.state_dict(), os.path.join(EXP_PATH, "model.pth"))


  1%|▏         | 2/157 [00:00<00:16,  9.23it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 157/157 [00:18<00:00,  8.29it/s]


Epoch 1/10 - Train Loss: 0.5041 Train Acc: 0.7537 - Test Loss: 1.7448 Test Acc: 0.4608


Epoch 2/10:   0%|          | 0/625 [00:00<?, ?it/s]


 STARTING FEATOUT 
 


100%|██████████| 157/157 [00:17<00:00,  8.83it/s]


...

Epoch 8/10 - Train Loss: 0.1895 Train Acc: 0.9154 - Test Loss: 1.2507 Test Acc: 0.5740


100%|██████████| 157/157 [00:17<00:00,  8.74it/s]


Epoch 9/10 - Train Loss: 0.1704 Train Acc: 0.9226 - Test Loss: 1.2610 Test Acc: 0.5834


100%|██████████| 157/157 [00:17<00:00,  8.78it/s]

Epoch 10/10 - Train Loss: 0.1635 Train Acc: 0.9255 - Test Loss: 1.2460 Test Acc: 0.5858


In [8]:
!zip -r Hers.zip logs/Standard_Featout


updating: logs/Standard_Featout/ (stored 0%)
updating: logs/Standard_Featout/logs.csv (deflated 51%)
updating: logs/Standard_Featout/model.pth (deflated 8%)
updating: logs/Standard_Featout/events.out.tfevents.1694711439.38d12c190597.4577.0 (deflated 58%)


In [7]:
from google.colab import files
files.download('Hers.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>